# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Load-libraries" data-toc-modified-id="Load-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load libraries</a></div><div class="lev1 toc-item"><a href="#Load-pre-trained-vgg-model-and-add-top-layers" data-toc-modified-id="Load-pre-trained-vgg-model-and-add-top-layers-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load pre-trained vgg model and add top layers</a></div><div class="lev1 toc-item"><a href="#Image-data-generators" data-toc-modified-id="Image-data-generators-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Image data generators</a></div><div class="lev1 toc-item"><a href="#Training---300x225" data-toc-modified-id="Training---300x225-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training - 300x225</a></div><div class="lev1 toc-item"><a href="#Training---600x450" data-toc-modified-id="Training---600x450-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Training - 600x450</a></div><div class="lev1 toc-item"><a href="#Training-convolutional-layers" data-toc-modified-id="Training-convolutional-layers-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Training convolutional layers</a></div><div class="lev1 toc-item"><a href="#Prediction" data-toc-modified-id="Prediction-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Prediction</a></div>

modified from https://www.kaggle.com/chmaxx/finetune-vgg16-0-97-with-minimal-effort

# Load libraries

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.vgg16 import VGG16

from keras_tqdm import TQDMNotebookCallback

from datetime import datetime
import os

import numpy as np
import pandas as pd
import math

pd.options.display.max_rows = 40

# Load pre-trained vgg model and add top layers

In [3]:
vgg16 = VGG16(weights = 'imagenet',include_top=False)

58900480/58889256 [==============================] - 34s 1us/step


In [4]:
vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [5]:
x = vgg16.get_layer('block5_conv3').output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)

model_final = Model(inputs=vgg16.input, outputs=x)

In [6]:
for layer in vgg16.layers:
    layer.trainable = False

model_final.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

# Image data generators

In [8]:
datagen = ImageDataGenerator( 
    featurewise_center            = True,
    rescale                       = 1.,
    rotation_range                = 10,
    width_shift_range             = .1,
    height_shift_range            = .1,
    shear_range                   = 0.2,
    zoom_range                    = 0.2,
    horizontal_flip               = True,
    vertical_flip                 = False,
    fill_mode                     = "reflect")

# normalization neccessary for correct image input to VGG16
datagen.mean=np.array([103.939, 116.779, 123.68], dtype=np.float32).reshape(1,1,3)

# no data augmentation for validation and test set
validgen = ImageDataGenerator(rescale=1., featurewise_center=True)
validgen.mean=np.array([103.939, 116.779, 123.68], dtype=np.float32).reshape(1,1,3)

In [9]:
# 600/450 _ 500/375 _ 400/300 _ 300/225

img_width  = 300
img_height = 225

train_data_dir      = "data/train"
validation_data_dir = "data/valid"
test_data_dir       = "data/test"

batch_size_train = 5
batch_size_val   = 15

train_gen = datagen.flow_from_directory(
        directory   = train_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_train,
        class_mode  = "binary",
        shuffle     = True)

val_gen = validgen.flow_from_directory(
        directory   = validation_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_val,
        class_mode  = "binary",
        shuffle     = False)

test_gen = validgen.flow_from_directory(
        directory   = test_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_val,
        class_mode  = "binary",
        shuffle     = False)

train_samples      = len(train_gen.filenames)
validation_samples = len(val_gen.filenames)
test_samples       = len(test_gen.filenames)

Found 1713 images belonging to 2 classes.
Found 582 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


# Training - 300x225

In [10]:
checkpoint = ModelCheckpoint("weights-iter-1-epoch-{epoch:02d}.hdf5",
                             monitor='val_acc',
                             verbose=0,
                             save_best_only=False,
                             save_weights_only=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

model_final.fit_generator(generator        = train_gen,
                          epochs           = 10, 
                          steps_per_epoch  = math.ceil(train_samples / batch_size_train), 
                          validation_data  = val_gen, 
                          validation_steps = math.ceil(validation_samples / batch_size_val), 
                          verbose          = 2,
                          callbacks        = [early_stopping, TQDMNotebookCallback(), checkpoint])

HBox(children=(IntProgress(value=0, description=u'Training', max=10), HTML(value=u'')))

Epoch 1/10


HBox(children=(IntProgress(value=0, description=u'Epoch 0', max=342), HTML(value=u'')))

 - 1634s - loss: 0.3109 - acc: 0.8840 - val_loss: 0.2244 - val_acc: 0.9088


ValueError: invalid literal for int() with base 10: '342.0'

# Training - 600x450

In [12]:
# 600/450 _ 500/375 _ 400/300 _ 300/225

img_width  = 600
img_height = 450

train_data_dir      = "data/train"
validation_data_dir = "data/valid"
test_data_dir       = "data/test"

batch_size_train = 5
batch_size_val   = 15

train_gen = datagen.flow_from_directory(
        directory   = train_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_train,
        class_mode  = "binary",
        shuffle     = True)

val_gen = validgen.flow_from_directory(
        directory   = validation_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_val,
        class_mode  = "binary",
        shuffle     = False)

test_gen = validgen.flow_from_directory(
        directory   = test_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_val,
        class_mode  = "binary",
        shuffle     = False)

train_samples      = len(train_gen.filenames)
validation_samples = len(val_gen.filenames)

Found 1713 images belonging to 2 classes.
Found 582 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [13]:
checkpoint = ModelCheckpoint("weights-iter-2-epoch-{epoch:02d}.hdf5",
                             monitor='val_acc',
                             verbose=0,
                             save_best_only=False,
                             save_weights_only=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

model_final.fit_generator(generator        = train_gen,
                          epochs           = 10, 
                          steps_per_epoch  = math.ceil(train_samples / batch_size_train), 
                          validation_data  = val_gen, 
                          validation_steps = math.ceil(validation_samples / batch_size_val), 
                          verbose          = 2,
                          callbacks        = [early_stopping, TQDMNotebookCallback(), checkpoint])

HBox(children=(IntProgress(value=0, description=u'Training', max=10), HTML(value=u'')))

Epoch 1/10


HBox(children=(IntProgress(value=0, description=u'Epoch 0', max=342), HTML(value=u'')))

KeyboardInterrupt: 

# Training convolutional layers

In [12]:
for i, layer in enumerate(model_final.layers):
   print(i, layer.name)

for layer in model_final.layers[:15]:
   layer.trainable = False
for layer in model_final.layers[15:]:
   layer.trainable = True

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 global_average_pooling2d_1
19 dense_1
20 dropout_1
21 dense_2


In [13]:
model_final.compile(optimizer = SGD(lr=0.0001, momentum=0.9, nesterov=True),
                    loss      = 'binary_crossentropy',
                    metrics   = ['accuracy'])

In [15]:
checkpoint = ModelCheckpoint("weights-iter-3-epoch-{epoch:02d}.hdf5",
                             monitor='val_acc',
                             verbose=0,
                             save_best_only=False,
                             save_weights_only=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

model_final.fit_generator(generator        = train_gen,
                          epochs           = 10, 
                          steps_per_epoch  = math.ceil(train_samples / batch_size_train), 
                          validation_data  = val_gen, 
                          validation_steps = math.ceil(validation_samples / batch_size_val), 
                          verbose          = 2,
                          callbacks        = [early_stopping, TQDMNotebookCallback(), checkpoint])

Epoch 1/10
287s - loss: 0.1699 - acc: 0.9399 - val_loss: 0.1338 - val_acc: 0.9718


Epoch 2/10
287s - loss: 0.1123 - acc: 0.9642 - val_loss: 0.1374 - val_acc: 0.9718


Epoch 3/10
287s - loss: 0.0688 - acc: 0.9769 - val_loss: 0.0914 - val_acc: 0.9718


Epoch 4/10
287s - loss: 0.0571 - acc: 0.9803 - val_loss: 0.1304 - val_acc: 0.9577


Epoch 5/10
287s - loss: 0.0636 - acc: 0.9786 - val_loss: 0.1062 - val_acc: 0.9683


Epoch 6/10
287s - loss: 0.0518 - acc: 0.9850 - val_loss: 0.0626 - val_acc: 0.9859


Epoch 7/10
287s - loss: 0.0373 - acc: 0.9908 - val_loss: 0.1168 - val_acc: 0.9683


Epoch 8/10
287s - loss: 0.0298 - acc: 0.9890 - val_loss: 0.0920 - val_acc: 0.9824


Epoch 9/10
287s - loss: 0.0259 - acc: 0.9913 - val_loss: 0.0991 - val_acc: 0.9806


Epoch 10/10
287s - loss: 0.0305 - acc: 0.9890 - val_loss: 0.0815 - val_acc: 0.9842
Epoch 00009: early stopping



# Prediction

In [19]:
test_gen = validgen.flow_from_directory(
        directory   = test_data_dir,
        target_size = (img_height, img_width),
        batch_size  = 32,
        class_mode  = "binary",
        shuffle     = False)
test_samples = len(test_gen.filenames)
preds = model_final.predict_generator(test_gen, math.ceil(test_samples / 32))

Found 1531 images belonging to 1 classes.


In [20]:
preds

array([[  1.28751271e-03],
       [  1.06840848e-03],
       [  1.57627165e-01],
       ..., 
       [  5.91619834e-02],
       [  2.59617136e-05],
       [  1.00000000e+00]], dtype=float32)

In [22]:
preds_filenames = test_gen.filenames

In [23]:
preds_filenames = [int(x.replace("unknown/", "").replace(".jpg", "")) for x in preds_filenames]

In [24]:
preds_filenames

[1098,
 718,
 326,
 1079,
 688,
 527,
 86,
 506,
 1047,
 211,
 1422,
 1306,
 1115,
 874,
 742,
 23,
 501,
 678,
 1399,
 1461,
 1039,
 936,
 994,
 1077,
 1358,
 487,
 341,
 704,
 746,
 20,
 1049,
 545,
 762,
 651,
 1424,
 1259,
 1466,
 220,
 225,
 332,
 269,
 919,
 277,
 251,
 842,
 992,
 1264,
 1099,
 911,
 1017,
 978,
 1391,
 888,
 405,
 1129,
 1505,
 810,
 1097,
 1244,
 363,
 1202,
 784,
 12,
 498,
 275,
 1366,
 826,
 879,
 1016,
 680,
 448,
 293,
 1239,
 1411,
 1048,
 34,
 1498,
 1400,
 657,
 347,
 1263,
 1413,
 668,
 352,
 135,
 645,
 1025,
 226,
 63,
 1112,
 1203,
 51,
 457,
 475,
 866,
 881,
 1297,
 904,
 1294,
 236,
 1439,
 856,
 474,
 1211,
 613,
 1242,
 443,
 599,
 887,
 411,
 154,
 1206,
 1469,
 555,
 640,
 681,
 314,
 1468,
 1345,
 245,
 706,
 392,
 664,
 632,
 1437,
 1166,
 1458,
 42,
 739,
 323,
 608,
 1061,
 597,
 32,
 210,
 955,
 132,
 109,
 639,
 551,
 156,
 284,
 1530,
 1287,
 1321,
 414,
 99,
 1230,
 1300,
 540,
 1508,
 113,
 1065,
 1364,
 45,
 1351,
 283,
 233,
 562,

In [25]:
df_result = pd.DataFrame({'name': preds_filenames, 'invasive': preds[:,0]})
df_result = df_result.sort_values("name")
df_result.index = df_result["name"]
df_result = df_result.drop(["name"], axis=1)

In [26]:
df_result

,invasive
name,
1,1.000000e+00
2,3.630535e-04
3,4.814070e-02
4,3.577948e-02
5,1.000000e+00
6,8.262738e-05
7,5.450361e-02
8,1.000000e+00
9,1.000000e+00


In [27]:
df_result.to_csv("submission_01.csv", encoding="utf8", index=True)

In [28]:
from IPython.display import FileLink
FileLink('submission_01.csv')

/home/ubuntu/extvol/kaggle-invasive-species-monitoring/submission_01.csv

In [ ]:
# Got 0.98680 on LB